# Distributed HPO with Ray Tune and XGBoost-Ray

This demo introduces **Ray tune's** key concepts using a a classification example. This example is derived from [Hyperparameter Tuning with Ray Tune and XGBoost-Ray](https://github.com/ray-project/xgboost_ray#hyperparameter-tuning). Basically, there are three basic steps or Ray Tune pattern for you as a newcomer to get started with using Ray Tune.

 1. Setup your config space and define your trainable and objective function
 2. Use tune to execute your training, supplying the appropriate arguments including: search space, [search algorithms](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#summary) or [trial schedulers](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-schedulers)
 3. Examine analyse the results
 
 <img src="https://docs.ray.io/en/latest/_images/tune-workflow.png" height="50%" width="60%">


See also the [Understanding Hyperparameter Tuning](https://github.com/anyscale/academy/blob/main/ray-tune/02-Understanding-Hyperparameter-Tuning.ipynb) notebook and the [Tune documentation](http://tune.io), in particular, the [API reference](https://docs.ray.io/en/latest/tune/api_docs/overview.html). 


In [1]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

import ray
from ray import tune
CONNECT_TO_ANYSCALE=True

In [2]:
if ray.is_initialized:
    ray.shutdown()
    if CONNECT_TO_ANYSCALE:
        ray.init("anyscale://jsd-weekly-demo")
    else:
        ray.init()

Authenticating
Loaded Anyscale authentication token from ANYSCALE_CLI_TOKEN.

Output
(anyscale +0.7s) WARNING: No working_dir specified! Files will only be uploaded to the cluster if a working_dir is provided or a project is detected. In the future, files will only be uploaded if working_dir is provided. To ensure files continue being imported going forward, set the working_dir in your runtime environment. See https://docs.ray.io/en/latest/handling-dependencies.html#runtime-environments.
(anyscale +0.4s) .anyscale.yaml found in project_dir. Directory is attached to a project.
(anyscale +0.5s) Using project (name: prj-weekly-demo, project_dir: /Users/jules/git-repos/ray-core-tutorial, id: prj_5rvR1w2ciyUs9RM27FeZ6FVB).
(anyscale +2.0s) cluster jsd-weekly-demo is currently running, the cluster will not be restarted.


2022-01-11 08:55:32,259	INFO packaging.py:352 -- Creating a file package for local directory '/Users/jules/git-repos/ray-core-tutorial'.
2022-01-11 08:55:32,302	INFO packaging.py:221 -- Pushing file package 'gcs://_ray_pkg_41ba73bd76ce0474.zip' (6.25MiB) to Ray cluster...
2022-01-11 08:55:37,161	INFO packaging.py:224 -- Successfully pushed file package 'gcs://_ray_pkg_41ba73bd76ce0474.zip'.


(anyscale +18.3s) Connected to jsd-weekly-demo, see: https://console.anyscale.com/projects/prj_5rvR1w2ciyUs9RM27FeZ6FVB/clusters/ses_GKnU1MGrXqZNmAkicHxSKU9G
(anyscale +18.3s) URL for head node of cluster: https://session-gknu1mgrxqznmakichxsku9g.i.anyscaleuserdata.com


## Step 1: Define a 'Trainable' training function to use with Ray Tune `ray.tune(...)`

In [3]:
NUM_OF_ACTORS = 4           # degree of parallel trials; each actor will have a separate trial
NUM_OF_CPUS_PER_ACTOR = 1   # number of CPUs per actor

ray_params = RayParams(num_actors=NUM_OF_ACTORS, cpus_per_actor=NUM_OF_CPUS_PER_ACTOR)

In [4]:
def train_func_model(config:dict, checkpoint_dir=None):
    # create the dataset
    train_X, train_y = load_breast_cancer(return_X_y=True)
    # Convert to RayDMatrix data structure
    train_set = RayDMatrix(train_X, train_y)

    # Empty dictionary for the evaluation results reported back
    # to tune
    evals_result = {}

    # Train the model with XGBoost train
    bst = train(
        params=config,                       # our hyperparameter search space
        dtrain=train_set,                    # our RayDMatrix data structure
        evals_result=evals_result,           # place holder for results
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)                # distributed parameters configs for Ray Tune

    bst.save_model("model.xgb")

## Step 2: Define a hyperparameter search space

In [5]:
 # Specify the typical hyperparameter search space
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

## Step 3: Run Ray tune main trainer and examine the results

Ray Tune will launch distributed HPO, using four remote actors, each with its own instance of the trainable func

<img src="images/ray_tune_dist_hpo.png" height="60%" width="70%"> 

In [ ]:
# Run tune
analysis = tune.run(
    train_func_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    verbose=1,
    resources_per_trial=ray_params.get_tune_resources()
)

(run pid=865) == Status ==
(run pid=865) Current time: 2022-01-11 09:00:17 (running for 00:00:00.21)
(run pid=865) Memory usage on this node: 2.6/62.0 GiB
(run pid=865) Using FIFO scheduling algorithm.
(run pid=865) Resources requested: 0/80 CPUs, 0/0 GPUs, 0.0/216.19 GiB heap, 0.0/92.37 GiB objects
(run pid=865) Result logdir: /home/ray/ray_results/train_func_model_2022-01-11_08-56-52
(run pid=865) Number of trials: 4/4 (4 PENDING)
(run pid=865) 
(run pid=865) 


(ImplicitFunc pid=3429) 2022-01-11 09:00:21,176	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=3429) 2022-01-11 09:00:23,296	INFO main.py:971 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=3505) [09:00:23] task [xgboost.ray]:140378505802224 got new rank 3
(_RemoteRayXGBoostActor pid=3503) [09:00:23] task [xgboost.ray]:140613502865856 got new rank 1
(_RemoteRayXGBoostActor pid=3504) [09:00:23] task [xgboost.ray]:140316667113968 got new rank 2
(_RemoteRayXGBoostActor pid=3502) [09:00:23] task [xgboost.ray]:140269844046272 got new rank 0


In [ ]:
print("Best hyperparameters", analysis.best_config)

In [ ]:
analysis.results_df.head(5)

---

In [ ]:
ray.shutdown()

## References

 * [Ray Train: Tune: Scalable Hyperparameter Tuning](https://docs.ray.io/en/master/tune/index.html)
 * [Introducing Distributed XGBoost Training with Ray](https://www.anyscale.com/blog/distributed-xgboost-training-with-ray)
 * [How to Speed Up XGBoost Model Training](https://www.anyscale.com/blog/how-to-speed-up-xgboost-model-training)
 * [XGBoost-Ray Project](https://github.com/ray-project/xgboost_ray)
 * [Distributed XGBoost on Ray](https://docs.ray.io/en/latest/xgboost-ray.html)